In [16]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
#Preproccessing 
def match_score(val):
    return re.findall('[0-9].0', val)[0]

df = pd.read_json('amazon_one_plus_reviews.json')
dataset = df[['review_rating', 'review_text']]
dataset['review_rating'] = dataset['review_rating'].apply(match_score).astype(float)

C:\Users\tarun\AppData\Local\Temp\ipykernel_30996\1790986702.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['review_rating'] = dataset['review_rating'].apply(match_score).astype(float)


,review_rating,review_text
0,5.0,"\n Yea..pre-ordered on 28 July, got it on 4 A..."
1,5.0,"\n Got it delivered yesterday , used for abou..."
2,5.0,\n An amazing phone!
3,5.0,\n Brilliant..
4,5.0,\n I was skeptical about changing from One pl...
...,...,...
30607,2.0,\n Quality of phone is great but from my pers...
30608,2.0,\n Not recommend\n
30609,2.0,\n Redmi and Amazon is engaged in a worst mar...
30610,2.0,"\n I am facing display retention problem, aft..."


In [26]:
dataset['review_rating'].value_counts()

5.0    13967
4.0     8537
3.0     4172
1.0     2088
2.0     1848
Name: review_rating, dtype: int64

In [19]:
#Standard BoW
"""
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english', ngram_range = (1,1), tokenizer = token.tokenize)
text_counts = cv.fit_transform(dataset['review_text'])
"""
#TFIDF
cv =TfidfVectorizer(max_df=0.90, min_df=2,max_features=1000,stop_words='english')
text_counts=cv.fit_transform(dataset['review_text'])

In [28]:
def score(predictions, test):
    correct = 0 
    for i in range(len(predictions)):
        if predictions[i] == test[test.index[i]]:
            correct += 1
    return(correct/len(predictions)) 

def distance_function(predictions, test):
    total_distance = 0
    total = 0
    for i in range(len(predictions)):
        distance = (predictions[i]  - test[test.index[i]])**2
        total_distance += distance
        total += 25
    return(1-(math.sqrt(total_distance/total)))

highest_accuracy = 0
highest_distance = 0
acc_vals = [0,0]
dis_vals = [0,0]
for i in [.5, .6, .7, .8, .9]:
    for j in [.01, .02, .03, .04, .05, .06, .07, .08, .09, .1]:
        cv =TfidfVectorizer(max_df=i, min_df=j,max_features=1000,stop_words='english')
        text_counts=cv.fit_transform(dataset['review_text'])
        X_train, X_test, Y_train, Y_test = train_test_split(text_counts, dataset['review_rating'], test_size = .25, random_state=5)
        model = MultinomialNB()
        model.fit(X_train, Y_train)
        predictions = model.predict(X_test)
        accuracy = score(predictions, Y_test)
        distance = distance_function(predictions, Y_test)
        if distance > highest_distance:
            highest_distance = distance
            dis_vals = [i,j]
        if accuracy > highest_accuracy:
            highest_accuracy = accuracy
            acc_vals = [i,j]
        """
        print("\nMax_df: " + str(i) + " | Min_df: " + str(j))
        print("\nAccuracy: " + str(accuracy*100))
        print("Distance: " + str(distance*100))
        print("\n")
        """
print("Highest Accuracy: " + str(highest_accuracy))
print("Max_df: " + str(acc_vals[0]) + " | Min_df: " + str(acc_vals[1]))
print("Highest Distance Score: " + str(highest_distance))
print("Max_df: " + str(dis_vals[0]) + " | Min_df: " + str(dis_vals[1]))



Highest Accuracy: 0.5093427414085979
Max_df: 0.5 | Min_df: 0.01
Highest Distance Score: 0.7429880513002021
Max_df: 0.5 | Min_df: 0.01


In [32]:

cv =TfidfVectorizer(max_df=0.5, min_df=0,max_features=1000, ngram_range = (1,3), stop_words='english')
text_counts=cv.fit_transform(dataset['review_text'])
X_train, X_test, Y_train, Y_test = train_test_split(text_counts, dataset['review_rating'], test_size = .25, random_state=5)

In [33]:
import math
def score(predictions, test):
    correct = 0 
    for i in range(len(predictions)):
        if predictions[i] == test[test.index[i]]:
            correct += 1
    return(correct/len(predictions)) 

def distance_function(predictions, test):
    total_distance = 0
    total = 0
    for i in range(len(predictions)):
        distance = (predictions[i]  - test[test.index[i]])**2
        total_distance += distance
        total += 25
    return(1-(math.sqrt(total_distance/total)))


basic_model = MultinomialNB()
basic_model.fit(X_train, Y_train)
predictions = basic_model.predict(X_test)
accuracy = score(predictions, Y_test)
print("-------------Bag of Words Classifer-------------")
print("Accuracy: " + str(accuracy*100))

-------------Bag of Words Classifer-------------
Accuracy: 52.593754083365994


In [34]:
print("-------------Only Single-------------")
cv = TfidfVectorizer(max_df=0.5, min_df=0,max_features=1000, ngram_range = (1,1), stop_words='english')
text_counts = cv.fit_transform(dataset['review_text'])
X_train, X_test, Y_train, Y_test = train_test_split(text_counts, dataset['review_rating'], test_size = .25, random_state=5)
model = MultinomialNB()
model.fit(X_train, Y_train)
predictions = model.predict(X_test)
accuracy = score(predictions, Y_test)
distance = distance_function(predictions, Y_test)
print("\nAccuracy: " + str(accuracy*100))
print("Distance: " + str(distance*100))
print("\n")
print("-------------Single & Bigram-------------")
cv = TfidfVectorizer(max_df=0.5, min_df=0,max_features=1000, ngram_range = (1,2), stop_words='english')
text_counts = cv.fit_transform(dataset['review_text'])
X_train, X_test, Y_train, Y_test = train_test_split(text_counts, dataset['review_rating'], test_size = .25, random_state=5)
model = MultinomialNB()
model.fit(X_train, Y_train)
predictions = model.predict(X_test)
accuracy = score(predictions, Y_test)
distance = distance_function(predictions, Y_test)
print("\nAccuracy: " + str(accuracy*100))
print("Distance: " + str(distance*100))
print("\n")
print("-------------Only Bigram-------------")
cv = TfidfVectorizer(max_df=0.5, min_df=0,max_features=1000, ngram_range = (2,2), stop_words='english')
text_counts = cv.fit_transform(dataset['review_text'])
X_train, X_test, Y_train, Y_test = train_test_split(text_counts, dataset['review_rating'], test_size = .25, random_state=5)
model = MultinomialNB()
model.fit(X_train, Y_train)
predictions = model.predict(X_test)
accuracy = score(predictions, Y_test)
distance = distance_function(predictions, Y_test)
print("\nAccuracy: " + str(accuracy*100))
print("Distance: " + str(distance*100))
print("\n")
print("-------------Single, Bigram, & Trigram-------------")
cv = TfidfVectorizer(max_df=0.5, min_df=0,max_features=1000, ngram_range = (1,3), stop_words='english')
text_counts = cv.fit_transform(dataset['review_text'])
X_train, X_test, Y_train, Y_test = train_test_split(text_counts, dataset['review_rating'], test_size = .25, random_state=5)
model = MultinomialNB()
model.fit(X_train, Y_train)
predictions = model.predict(X_test)
accuracy = score(predictions, Y_test)
distance = distance_function(predictions, Y_test)
print("\nAccuracy: " + str(accuracy*100))
print("Distance: " + str(distance*100))
print("\n")
print("-------------Bigram & Trigram-------------")
cv = TfidfVectorizer(max_df=0.5, min_df=0,max_features=1000, ngram_range = (2,3), stop_words='english')
text_counts = cv.fit_transform(dataset['review_text'])
X_train, X_test, Y_train, Y_test = train_test_split(text_counts, dataset['review_rating'], test_size = .25, random_state=5)
model = MultinomialNB()
model.fit(X_train, Y_train)
predictions = model.predict(X_test)
accuracy = score(predictions, Y_test)
distance = distance_function(predictions, Y_test)
print("\nAccuracy: " + str(accuracy*100))
print("Distance: " + str(distance*100))
print("\n")
print("-------------Only Trigram-------------")
cv = TfidfVectorizer(max_df=0.5, min_df=0,max_features=1000, ngram_range = (3,3), stop_words='english')
text_counts = cv.fit_transform(dataset['review_text'])
X_train, X_test, Y_train, Y_test = train_test_split(text_counts, dataset['review_rating'], test_size = .25, random_state=5)
model = MultinomialNB()
model.fit(X_train, Y_train)
predictions = model.predict(X_test)
accuracy = score(predictions, Y_test)
distance = distance_function(predictions, Y_test)
print("\nAccuracy: " + str(accuracy*100))
print("Distance: " + str(distance*100))
print("\n")

-------------Only Single-------------

Accuracy: 51.91428198092252
Distance: 76.55781303669876


-------------Single & Bigram-------------

Accuracy: 52.52842022736181
Distance: 77.17461278927028


-------------Only Bigram-------------

Accuracy: 49.37932836796028
Distance: 73.93531952480753


-------------Single, Bigram, & Trigram-------------

Accuracy: 52.593754083365994
Distance: 77.1929390766173


-------------Bigram & Trigram-------------

Accuracy: 49.22252711355024
Distance: 73.78435775955155


-------------Only Trigram-------------

Accuracy: 46.38703776296877
Distance: 69.96539301703895




array([5., 4., 4., ..., 4., 4., 5.])

Int64Index([25605,  5660,  8949, 25536,  8441, 22645, 18013,   868,  7155,
             7462,
            ...
              611, 10064,  1026, 21386,  2457,  2519, 10211,  8977, 15067,
            12448],
           dtype='int64', length=7653)